In [19]:
from dotenv import load_dotenv
load_dotenv()
import os
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
import numpy as np
print(np.__version__)


1.24.3


In [20]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(name = "rag", dimension = 384, metric="cosine", spec = ServerlessSpec(cloud="aws", region="us-east-1"))

In [21]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Carter',
  'subject': 'Computer Science',
  'stars': 4,
  'review': 'Dr. Carter is knowledgeable and approachable, but her exams are tough.'},
 {'professor': 'Dr. John Smith',
  'subject': 'Mathematics',
  'stars': 5,
  'review': 'Excellent professor, explains concepts very clearly and is always available for help.'},
 {'professor': 'Dr. Alice Johnson',
  'subject': 'Physics',
  'stars': 3,
  'review': 'Dr. Johnson is good but sometimes goes too fast during lectures.'},
 {'professor': 'Dr. Robert Brown',
  'subject': 'Chemistry',
  'stars': 2,
  'review': "Dr. Brown's lectures are a bit disorganized, and the labs are very challenging."},
 {'professor': 'Dr. Michael Davis',
  'subject': 'Biology',
  'stars': 5,
  'review': 'Dr. Davis is passionate about the subject and makes the class enjoyable.'},
 {'professor': 'Dr. Sarah Wilson',
  'subject': 'English',
  'stars': 4,
  'review': 'Engaging lectures with plenty of class discussions, but grading is tough.'},
 {

In [22]:
processed_data = []
# Initialize Hugging Face Sentence Transformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

for review in data['reviews']:
    embedding = model.encode(review['review']).tolist()
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [23]:
processed_data[0]

{'values': [0.020304301753640175,
  -0.012235920876264572,
  -0.017441779375076294,
  0.0560610368847847,
  -0.08772089332342148,
  0.0044730789959430695,
  -0.008886693976819515,
  0.016050904989242554,
  -0.06884456425905228,
  0.08571860939264297,
  -0.096074178814888,
  0.049738556146621704,
  -0.07081449776887894,
  0.015036063268780708,
  -0.10181303322315216,
  -0.025240181013941765,
  -0.007768827490508556,
  -0.04903560131788254,
  0.03085356391966343,
  -0.017512336373329163,
  -0.08566844463348389,
  0.0519479364156723,
  0.03494575247168541,
  0.06523554027080536,
  0.013986092060804367,
  -0.014981858432292938,
  -0.04093986377120018,
  -0.04602888971567154,
  0.10546157509088516,
  -0.027549786493182182,
  -0.05343099683523178,
  0.020962925627827644,
  -0.045994654297828674,
  0.08761782944202423,
  0.035643987357616425,
  0.03881826251745224,
  0.01613120175898075,
  0.09735375642776489,
  -0.030530113726854324,
  -0.004896653816103935,
  0.049421973526477814,
  -0.0325

In [24]:
index = pc.Index("rag")
index.upsert(
    vectors = processed_data,
    namespace = "ns1"
)

{'upserted_count': 20}

In [25]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}